In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from fbprophet import Prophet 
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
import itertools
from fbprophet.diagnostics import performance_metrics
import tensorflow as tf
from tensorflow import keras
from catboost import CatBoostRegressor

In [2]:
#Lendo e transformando o arquivo CSV em um objeto pandas DataFrame nomeado de "df"
dff = pd.read_csv('C:/Users/juan_/Documents/GitHub/Datasets/demand.csv')

In [3]:
#Verificando o head e tail do CSV
dff

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
...,...,...,...,...,...
1048570,Product_1791,Whse_J,Category_006,2016/4/27,1000
1048571,Product_1974,Whse_J,Category_006,2016/4/27,1
1048572,Product_1787,Whse_J,Category_006,2016/4/28,2500
1048573,Product_0901,Whse_J,Category_023,2016/10/7,50


In [4]:
#Analisando algumas informações a respeito do DataFrame
dff.describe()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
count,1048575,1048575,1048575,1037336,1048575
unique,2160,4,33,1729,3828
top,Product_1359,Whse_J,Category_019,2013/9/27,1000
freq,16936,764447,481099,2075,112682


In [5]:
#Verificando a existência de dados não preenchidos no DataFrame
dff.isnull().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

In [6]:
#Verificando a relação de dados faltantes com o total de dados
x = (11239/1048575)*100
print(f'Os dados faltantes equivalem à {round(x,3)}% do total dos dados do DataFrame')

Os dados faltantes equivalem à 1.072% do total dos dados do DataFrame


In [7]:
#Os dados não preenchidos beiram 1% do total dos dados, sendo assim, resolvi dropar os dados. 
#Outro motivo para dropar os dados é que não era possível estimar os dados faltantes.

#Dropando as linhas dos dados faltantes e atualizando o DF.
dff.dropna(inplace=True)
dff.isnull().sum()

Product_Code        0
Warehouse           0
Product_Category    0
Date                0
Order_Demand        0
dtype: int64

In [8]:
#Não está no notebook, mas alguns números possuíam parenteses, o código abaixo é para retirar esses parenteses (servirá lá na frente)
dff['Order_Demand']=dff['Order_Demand'].str.replace('(','',regex=True)
dff['Order_Demand']=dff['Order_Demand'].str.replace(')','',regex=True)

In [9]:
#O código abaixo separa os valores das datas, e após isso, cada valor é atribuído a uma nova coluna no DataFrame
dff[['Year','Month','Day']] = dff['Date'].str.split("/", expand = True)
dff[['Month']] = dff[['Month']].astype(int)
dff[['Year']] = dff[['Year']].astype(int)
#df[['Day']] = df[['Day']].astype(int)

#A retirada dos parenteses dos números foi necessário na hora de converter a coluna "Order_Demand" de str para int
dff[['Order_Demand']] = dff[['Order_Demand']].astype(int)

In [10]:
#Ordenando o DataFrame pela Data
dff['Test'] = dff['Year'].map(str) +'/'+ dff['Month'].map(str)
dff['Test'] = pd.to_datetime(dff['Test'])
dff.sort_values('Date', inplace=True)
dff.reset_index(inplace=True)
dff.drop('index',axis=1,inplace=True)
dff

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Year,Month,Day,Test
0,Product_0965,Whse_A,Category_006,2011/1/8,2,2011,1,8,2011-01-01
1,Product_0412,Whse_S,Category_007,2011/10/20,2,2011,10,20,2011-10-01
2,Product_0125,Whse_S,Category_011,2011/10/20,2,2011,10,20,2011-10-01
3,Product_0642,Whse_C,Category_019,2011/10/31,3,2011,10,31,2011-10-01
4,Product_2137,Whse_S,Category_009,2011/11/18,25,2011,11,18,2011-11-01
...,...,...,...,...,...,...,...,...,...
1037331,Product_0689,Whse_J,Category_011,2017/1/6,200,2017,1,6,2017-01-01
1037332,Product_1970,Whse_J,Category_005,2017/1/6,2000,2017,1,6,2017-01-01
1037333,Product_1904,Whse_C,Category_019,2017/1/9,4000,2017,1,9,2017-01-01
1037334,Product_0471,Whse_C,Category_015,2017/1/9,30,2017,1,9,2017-01-01


In [11]:
#Verificando a quantidade de registros de cada armazens
dff[['Warehouse']].value_counts()

Warehouse
Whse_J       764447
Whse_A       142335
Whse_S        88200
Whse_C        42354
dtype: int64

In [12]:
#Analisando os dados do DataFrame original
xdf = dff['Order_Demand'].sum()
print(dff.nunique())
print(f'\nTotal of Order Demands: {xdf}')

Product_Code        2160
Warehouse              4
Product_Category      33
Date                1729
Order_Demand        3309
Year                   7
Month                 12
Day                   31
Test                  68
dtype: int64

Total of Order Demands: 5134173931


In [13]:
df_prod = dff[['Product_Code','Order_Demand','Date','Month','Year','Test']]
df_prod
df_prod[['Product_Code']].value_counts()

Product_Code
Product_1359    16936
Product_1295    10575
Product_1378     9770
Product_0620     9428
Product_1286     8888
                ...  
Product_1698        2
Product_1703        1
Product_0465        1
Product_2099        1
Product_0853        1
Length: 2160, dtype: int64

In [14]:
df = df_prod[(df_prod['Product_Code']=='Product_1295')]
#df['Test'] = df['Year'].map(str) +'/'+ df['Month'].map(str)
df

,Product_Code,Order_Demand,Date,Month,Year,Test
686,Product_1295,60000,2012/1/10,1,2012,2012-01-01
818,Product_1295,1000,2012/1/10,1,2012,2012-01-01
925,Product_1295,30000,2012/1/10,1,2012,2012-01-01
1014,Product_1295,1000,2012/1/10,1,2012,2012-01-01
1188,Product_1295,12000,2012/1/10,1,2012,2012-01-01
...,...,...,...,...,...,...
1036593,Product_1295,10000,2016/9/8,9,2016,2016-09-01
1036651,Product_1295,300,2016/9/9,9,2016,2016-09-01
1036842,Product_1295,30000,2016/9/9,9,2016,2016-09-01
1037058,Product_1295,10000,2016/9/9,9,2016,2016-09-01


In [15]:
#df['Date'] = pd.to_datetime(df['Date'])

In [16]:
df = df.groupby(by=['Test']).sum()
df.drop(columns=['Month','Year'],inplace=True)

In [17]:
df.reset_index(inplace=True)

In [18]:
#df['Date'] = pd.to_datetime(df['Test'])
#df

In [19]:
df['ds'] = df['Test']
df['y'] = df['Order_Demand']
df.drop(columns=['Test','Order_Demand'],inplace=True)

In [20]:
size = len(df) - 2
size

58

In [21]:
train=df[:size]
test=df
test

,ds,y
0,2012-01-01,1360800
1,2012-02-01,1863200
2,2012-03-01,2190800
3,2012-04-01,1817400
4,2012-05-01,1330700
5,2012-06-01,1676100
6,2012-07-01,1368000
7,2012-08-01,1396600
8,2012-09-01,1420500
9,2012-10-01,1900300


In [29]:
model=CatBoostRegressor(iterations=100, depth=10, learning_rate=0.03, loss_function='RMSE')
m = model.fit(train['ds'], train['y'])

0:	learn: 368768.7862999	total: 1.27ms	remaining: 126ms
1:	learn: 366045.1769934	total: 5.63ms	remaining: 276ms
2:	learn: 362717.0668890	total: 10.3ms	remaining: 334ms
3:	learn: 360289.0026172	total: 13.4ms	remaining: 323ms
4:	learn: 357536.1973076	total: 16.2ms	remaining: 308ms
5:	learn: 354651.5398078	total: 17.8ms	remaining: 279ms
6:	learn: 351928.7956835	total: 18.6ms	remaining: 247ms
7:	learn: 349892.4493303	total: 21.3ms	remaining: 245ms
8:	learn: 347039.3510023	total: 24.2ms	remaining: 245ms
9:	learn: 344458.5679225	total: 25.3ms	remaining: 227ms
10:	learn: 342227.4395094	total: 25.7ms	remaining: 208ms
11:	learn: 339751.1952093	total: 26.3ms	remaining: 193ms
12:	learn: 337773.5074340	total: 26.7ms	remaining: 179ms
13:	learn: 336120.0137941	total: 30ms	remaining: 184ms
14:	learn: 334384.8391461	total: 30.4ms	remaining: 172ms
15:	learn: 332949.7497172	total: 33.1ms	remaining: 174ms
16:	learn: 330960.6204473	total: 34.5ms	remaining: 168ms
17:	learn: 328289.3023878	total: 37.3ms	rem

In [34]:
CatBoostRegressor.predict(data=test,
        prediction_type=None,
        ntree_start=0,
        ntree_end=0,
        thread_count=-1,
        verbose=None)

TypeError: predict() missing 1 required positional argument: 'self'